# MNLP Homework 2 - OCRed text cleaning

In [ ]:
import google.generativeai as genai
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import json
import nltk
import re
import os


from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from nltk.tokenize import sent_tokenize
from huggingface_hub import login
from dotenv import load_dotenv
from rouge import Rouge
from tqdm import tqdm


### Environment setting

In [ ]:
device = torch.device("cuda")
print(torch.cuda.get_device_name(0))
print("Supports float16:", torch.cuda.is_available())
print("Supports bfloat16:", torch.cuda.is_bf16_supported())

Be sure to either set a .env file or put here the keys for gemini and hugging face 

In [ ]:
if '.env' in os.listdir(os.getcwd()):
    load_dotenv()
    CACHE_SUBPATH = os.environ['CACHE_PATH']
    TEXTS_SUBPATH = os.environ['DATA_PATH']
    OUTPUT_SUBPATH = os.environ['OUTPUT_PATH']
    HF_TOKEN = os.environ['HF_TOKEN']
    GEMINI_KEY = os.environ['GEMINI_KEY']
    
else:
    CACHE_SUBPATH = 'cache'
    TEXTS_SUBPATH = 'data\ita'
    OUTPUT_SUBPATH = 'cleaned'
    HF_TOKEN = ''
    GEMINI_KEY = ''

os.environ["ACCELERATE_USE_TORCH_DEVICE"] = "true"

TEXTS_PATH = os.path.join(os.getcwd(), TEXTS_SUBPATH)
OUTPUT_PATH =os.path.join(os.getcwd(), OUTPUT_SUBPATH)
CACHE_PATH = os.path.join(os.getcwd(), CACHE_SUBPATH)

os.makedirs(TEXTS_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

os.environ['HF_HOME'] = CACHE_PATH

FILE_LIST = ["original_ocr.json", "cleaned.json"]
OUTPUT_PREFIX = "Pizza_Language_&_Mandolino-hw2_ocr"

assert HF_TOKEN != '', "No key for hugging face"
login(token=HF_TOKEN)
assert GEMINI_KEY != '', "No key for gemini"
genai.configure(api_key=GEMINI_KEY) 

### Retrieval texts

In [ ]:
datasetRaw = {}

for name in FILE_LIST:
    if name not in os.listdir(TEXTS_PATH):
        print(f"File [{name}] not Found in directory [{TEXTS_PATH}]")

    file_path = os.path.join(TEXTS_PATH, name)
    with open(file_path, 'r') as file:
        datasetRaw[name.split('.')[0]] = json.load(file)
        file.close()

### Utils functions definition

In [ ]:
def smart_split_paragraphs(text:str)->list[str]:
    text = re.sub(r'\s+', ' ', text.strip())

    text = re.sub(r'\.{2,}', lambda m: f"<DOTS{len(m.group(0))}>", text)

    fragments = re.split(r'(?<!\.)\.(?!\.)(?=\s|$)', text)

    result = []
    for frag in fragments:
        frag = frag.strip()
        if not frag:
            continue
        frag = re.sub(r'<DOTS(\d+)>', lambda m: '.' * int(m.group(1)), frag)
        result.append(frag + ". ")

    return result

In [ ]:
def merge_hyphenated_words(text:str)->str:
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    return text

##### Preprocessing example

In [ ]:
sample = datasetRaw['original_ocr']['1']
sentences = smart_split_paragraphs(sample)

for i in range(len(sentences)):
    sentences[i] = merge_hyphenated_words(sentences[i])

for i, s in enumerate(sentences, 1):
    print(f"{s}")

In [ ]:
def correct_text(texts: list[str], model_obj: dict)-> list[dict]:
    results = []
    for text in texts:
        
        prompt = model_obj['prompt_gen'](texts)
        
        output = model_obj['generator'](
            prompt,
            max_new_tokens=len(model_obj['tokenizer'].encode(text)) - 2,
            do_sample=False,
            top_p=1.0,
            num_beams=4,
            repetition_penalty=1.1,
        )[0]["generated_text"]

        cleaned = output.split("Testo corretto:")[-1].strip().split("\n")[0]

        print(f"Input:  {text}")
        print(f"Output: {cleaned}\n")
        
        results.append({"input": text, "output": cleaned})

    return results

In [ ]:
def execute_experiment(model_obj:dict) -> None:
    print(model_obj['name'])
    for chapter, sample in datasetRaw['original_ocr'].items():
        print(f"Chapter {chapter}")
        sentences = smart_split_paragraphs(sample)
        for i in range(len(sentences)):
            sentences[i] = merge_hyphenated_words(sentences[i])

        corrected_data = correct_text(sentences, model_obj)

        with open(os.path.join(OUTPUT_PATH,model_obj['name'],f"Chapter_{chapter}.json"), "w+", encoding="utf-8") as file:
            json.dump(corrected_data, file, ensure_ascii=False)
    return

In [ ]:
def reformat_output(model_name:str, prefix:str=OUTPUT_PREFIX)->None:
    cleaned_txt = {}
    dest_path = os.path.join(OUTPUT_PATH, model_name)
    for i, file_name in enumerate(os.listdir(dest_path)):
        with open(os.path.join(dest_path,file_name), 'r', encoding='utf-8') as file:
            data = json.load(file)
            text = ' '.join(item['output'] for item in data)
            cleaned_txt[str(i+1)] = text
            file.close()

    fname = f"{prefix}-{model_name}.json"
    with open(os.path.join(OUTPUT_PATH, fname), 'w', encoding='utf-8') as f_out:
        json.dump(cleaned_txt, f_out, ensure_ascii=False, indent=2)
        f_out.close()

## Minerva

In [ ]:
MINERVA_VERSION = "sapienzanlp/Minerva-3B-base-v1.0"
minerva_tok = AutoTokenizer.from_pretrained(MINERVA_VERSION, trust_remote_code=True)
minerva_model = AutoModelForCausalLM.from_pretrained(MINERVA_VERSION, 
                                            trust_remote_code=True, 
                                            device_map=device, 
                                            torch_dtype=torch.float16
                                            )

minerva_gen = pipeline("text-generation", 
                        model=minerva_model, 
                        tokenizer=minerva_tok, 
                        pad_token_id=minerva_tok.eos_token_id
                    )

In [ ]:
minerva_obj = {
    "model":minerva_model,
    "generator": minerva_gen,
    "tokenizer":minerva_tok,
    "name":"Minerva",
    "prompt_gen": lambda x: (
            f"Sei un generatore di eBooks. "
            f"Correggi TUTTI gli errori OCR nel testo che ti verrà fornito. "
            f"Non aggiungere, togliere o cambiare parole. "
            f"Non modificare la punteggiatura esistente. "
            f"Non interpretare, riassumere o riscrivere il testo. "
            f"Mantieni esattamente lo stesso numero di parole e l'ordine delle parole. "
            f"Il testo corretto deve conservare il significato originale inalterato. "
            f"La tua unica funzione è la correzione di errori OCR: lettere errate, parole spezzate, ed errori di capitalizzazioni. "
            f"Rendi maiuscole le iniziali di frase e i nomi propri. Rendi minuscole le parole che non sono nomi propri o inizi di frase ma sono in maiuscolo. "
            f"Di seguito sono forniti esempi di input OCR e il corrispondente output corretto. Apprendi da questi esempi per svolgere il tuo compito.\n"
            f"Esempi di correzione OCR (inclusa capitalizzazione):\n"
            f"Input:  'qvesto è un testo. sembra ocr.'\n"
            f"Output: 'Questo è un testo. Sembra OCR.'\n"
            f"Input:  'l'aquila vola alta. il cieto è blù.'\n"
            f"Output: 'L'aquila vola alta. Il cielo è blu.'\n"
            f"Input: 'la matita è rota. c'è del tem po.'\n"
            f"Output: 'La matita è rotta. C'è del tempo.'\n"
            f"Input: 'i1 libro è su1 tavolo. LA STAMPA è chiara.'\n"
            f"Output: 'Il libro è sul tavolo. La stampa è chiara.'\n"
            f"Input: 'abbiam o un gTande paeco. la vitta è bella.'\n"
            f"Output: 'Abbiamo un grande pacco. La vita è bella.'\n"
            f"Testo OCR: {x} "
            f"Testo corretto:"
        )
}

In [ ]:
execute_experiment(minerva_obj)
reformat_output(minerva_obj['name'])

### Llama

In [ ]:
LLAMA_VERSION = "meta-llama/Llama-3.2-3B"

llama_tok = AutoTokenizer.from_pretrained(LLAMA_VERSION)

llama_model = AutoModelForCausalLM.from_pretrained(
    LLAMA_VERSION,
    device_map="auto", 
    torch_dtype="auto"
)

llama_gen = pipeline(
    "text-generation",
    model=llama_model,
    tokenizer=llama_tok
)

In [ ]:
llama_obj = {
    "model":llama_model,
    "generator":llama_gen,
    "tokenizer":llama_tok,
    "name":"Llama",
    "prompt_gen": lambda x: (
            f"Sei un Grillo Parlante! "
            f"Correggi TUTTI gli errori OCR nel testo che ti verrà fornito. "
            f"Non aggiungere, togliere o cambiare parole. "
            f"Non modificare la punteggiatura esistente. "
            f"Non interpretare, riassumere o riscrivere il testo. "
            f"Mantieni esattamente lo stesso numero di parole e l'ordine delle parole. "
            f"Il testo corretto deve conservare il significato originale inalterato. "
            f"La tua unica funzione è la correzione di errori OCR: lettere errate, parole spezzate, ed errori di capitalizzazioni. "
            f"Rendi maiuscole le iniziali di frase e i nomi propri. Rendi minuscole le parole che non sono nomi propri o inizi di frase ma sono in maiuscolo. "
            f"Di seguito sono forniti esempi di input OCR e il corrispondente output corretto. Apprendi da questi esempi per svolgere il tuo compito.\n"
            f"Esempi di correzione OCR (inclusa capitalizzazione):\n"
            f"Input: 'qvesto è un testo. sembra ocr.'\n"
            f"Output: 'Questo è un testo. Sembra OCR.'\n"
            f"Input: 'l'aquila vola alta. il cieto è blù.'\n"
            f"Output: 'L'aquila vola alta. Il cielo è blu.'\n"
            f"Input: 'la matita è rota. c'è del tem po.'\n"
            f"Output: 'La matita è rotta. C'è del tempo.'\n"
            f"Input: 'i1 libro è su1 tavolo. LA STAMPA è chiara.'\n"
            f"Output: 'Il libro è sul tavolo. La stampa è chiara.'\n"
            f"Input: 'abbiam o un gTande paeco. la vitta è bella.'\n"
            f"Output: 'Abbiamo un grande pacco. La vita è bella.'\n"
            f"Input: 'rn'\n"
            f"Output: 'm'\n"
            f"Input: 'Ko'\n"
            f"Output: 'No'\n"
            f"Input: '1ibro'\n"  
            f"Output: 'libro'\n"
            f"Input: '0cchio'\n" 
            f"Output: 'occhio'\n"
            f"Input: 'cl'\n" 
            f"Output: 'd'\n"
            f"Input: 'e, '\n"
            f"Output: 'e '\n"
            f"Testo OCR: {x} "
            f"Testo corretto:"
        )
}

In [ ]:
execute_experiment(llama_obj)
reformat_output(llama_obj['name'])

### Evaluation

In [ ]:
GEMINI_VERSION = "gemini-1.5-flash"

def segment_wrapper(segment_ocred, segment_clean):
    return f"""
        ### Task: Il primo testo è una versione del secondo estratto con oc-Red, ed è stata processata per ridurre gli errori, valuta con un punteggio tra 1 e 100 tenendo conto di correttezza, comprensibilità e somiglianza
        ### Testo da valutare: {segment_ocred}.
        ### Testo di confronto: {segment_clean}.
        ### Requisiti:
            - Scrivi il risultato in formato <criterio>:<punteggio>.
            - Non usare altri numeri interi o fai altri commenti
        ### Risultato:
    """

In [ ]:
rouge_eval = {}
gemini_eval = {}
rouge_baseline = {}
gemini_baseline = {}


for name in [llama_obj['name'], minerva_obj['name']]:
    model_scorer = Rouge()
    gemini = genai.GenerativeModel(model_name=GEMINI_VERSION)
    filename = os.path.join(OUTPUT_PATH, f"{OUTPUT_PREFIX}-{name}.json")
    gemini_eval[name] = [] 
    
    baseline_set = []
    reference_set = []
    produced_set = []
    
    with open(filename, "r", encoding='utf-8') as file_desc:
        output_log = json.load(file_desc)
        file_desc.close()
    
    for k,v in output_log.items():
        produced_set.append(v)
        reference_set.append(datasetRaw['cleaned'][f"{k+1}"])
        if len(baseline_set) >= int(k)-1:
            baseline_set.append(datasetRaw['original_ocr'][f"{k+1}"])
    
        gemini_input_eval = segment_wrapper(v, datasetRaw['cleaned'][f"{k+1}"])
        evaluation_gemini = gemini.generate_content(gemini_input_eval)
        gemini_eval[name].append(evaluation_gemini.text)
        
    rouge_eval[name] = model_scorer.get_scores(produced_set,reference_set)
    rouge_baseline[name] = model_scorer.get_scores(baseline_set, reference_set)
    
print(f"{'-'*20}")
print(rouge_eval)
print(rouge_baseline)
print(f"{'-'*20}")

In [ ]:
def rouge_formatter(eval_list:list)-> JSON:
    res = {}
    for n, score in enumerate(eval_list):
        res[f"{n+1}"] = {
            "rouge-1": score['rouge-1']['f'],
            "rouge-2": score['rouge-2']['f'],
            "rouge-l": score['rouge-l']['f'],
        }
    return res

def gemini_formatter(eval_list:list)-> JSON:
    res = {}
    for n, sentence in enumerate(eval_list):
        scores = sentence.strip().split('\n')
        entry = {}
        for s in scores:
            s = s.split(":")
            entry[f"{s[0]}"] = int(s[1])/100
        res[f"{n+1}"] = entry
    return res

In [ ]:
for n in [llama_obj['name'], minerva_obj['name']]:
    with open(os.path.join(OUTPUT_PATH, f"{OUTPUT_PREFIX}-{n}_geminiEval.json"),"w") as file:
        json.dump(gemini_formatter(gemini_eval[n]))
        file.close() 

In [ ]:
for n in [llama_obj['name'], minerva_obj['name']]:
    with open(os.path.join(OUTPUT_PATH, f"{OUTPUT_PREFIX}-{n}_rouge_f1.json"),"w") as file:
        json.dump(rouge_formatter(gemini_eval[n]))
        file.close() 